線形回帰であるリッジ回帰、LASSO、Elastic Netを実行してみる。

In [1]:
import pandas as pd
import sys

from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score


print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


### 1. データの準備

Toyデータを用意する。  
[make_regression](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html)関数を使うと自身が望むような回帰分析の検証のためのサンプルデータを用意することができる。  
今回は`サンプル数=1000`, `説明変数の数=10`, そして再現性を担保するために`random_state=0`, `shuffle=False`とした。

In [2]:
X, y = make_regression(n_samples=1000, n_features=10,
                                                random_state=0, shuffle=False)
X = pd.DataFrame(X)

In [3]:
X.head()

,0,1,2,3,4,5,6,7,8,9
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740


In [4]:
y[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [5]:
X.shape

(1000, 10)

---

### 2. リッジ回帰

#### 2.1 機械学習

[Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)クラスのコンストラクタの引数`alpha`が講義資料の`λ`に相当する。今回は`alpha=1`として学習を行ってみる。

In [6]:
regr = Ridge(alpha=1.0)
regr.fit(X, y) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

#### 2.2 説明変数の重み、係数

説明変数の重みは`coef_`、係数は`intercept_`に保存されている。[参考ドキュメント](http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [7]:
regr.coef_

array([  9.56477279,  40.1465737 ,  52.49713485,  73.14621686,
        13.51119737,  43.97620616,  69.47748865,  63.21789581,
         8.40281783,  33.60209855])

In [8]:
regr.intercept_

-0.010409753984825798

#### 2.3 予測および性能評価

予測値を求めるには`predict`メソッドを用いる。

In [9]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 299.84797707,  194.04108457,   13.21386842,  -24.58870742,
        -98.68132918])

また`score`メソッドを用いることで実測値と予測値の間の決定係数(r<sup>2</sup>)を求めることができる。

In [10]:
regr.score(X, y)

0.99999866516220048

#### 2.4 ペナルティ項の重み(引数alpha, 講義資料ではλ)を変えてみる

`alpha=0.5`とし説明変数の重みや決定係数が変化するか確認してみる。

In [11]:
regr = Ridge(alpha=0.5)
regr.fit(X, y) 

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
regr.coef_

array([  9.56572052,  40.16931242,  52.5260632 ,  73.18969112,
        13.51953007,  43.9996237 ,  69.52131158,  63.25331236,
         8.40325475,  33.62287967])

In [13]:
regr.intercept_

-0.0052077530301595232

In [14]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.02712346,  194.15890573,   13.23569097,  -24.59574554,
        -98.72095804])

In [15]:
regr.score(X, y)

0.99999966589939004

わずかであるが説明変数の重みが変化したり、決定係数が大きくなったりした。

#### 2.5 K-fold cross validation

[cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)関数を用いた10-fold cross validationを行い、モデルの性能評価を行う時は以下のようにスクリプトを書けば良い。

In [16]:
regr = Ridge()
cross_val_score(regr, X, y, cv=10)

array([ 0.99999832,  0.99999826,  0.99999836,  0.9999983 ,  0.9999982 ,
        0.99999826,  0.99999839,  0.99999831,  0.99999832,  0.99999827])

このようにcross validationを行う時に、以下のように`λ`を変化させて性能の差を比較し、  
最もよい性能の時の`λ`を採用するという判断もできる。

In [17]:
alphas = [i / 10 for i in range(0, 11)]
alphas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [18]:
for alpha in alphas:
    regr = Ridge(alpha=alpha)
    scores = cross_val_score(regr, X, y, cv=10)    
    print(f'alpha={alpha}, score_mean={scores.mean()}')

alpha=0.0, score_mean=1.0
alpha=0.1, score_mean=0.9999999829464568
alpha=0.2, score_mean=0.9999999318038253
alpha=0.3, score_mean=0.9999998465990861
alpha=0.4, score_mean=0.9999997273591988
alpha=0.5, score_mean=0.9999995741111013
alpha=0.6, score_mean=0.9999993868817096
alpha=0.7, score_mean=0.9999991656979187
alpha=0.8, score_mean=0.9999989105866021
alpha=0.9, score_mean=0.9999986215746117
alpha=1.0, score_mean=0.9999982986887783


今回の例では、`alpha=1`のときが最も性能が良い。

---

### 3. LASSO

#### 3.1 機械学習

[Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)クラスのコンストラクタの引数`alpha`が講義資料の`λ`に相当する。今回は`alpha=1`として学習を行ってみる。

In [19]:
regr = Lasso(alpha=1.0)
regr.fit(X, y) 

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

#### 3.2 説明変数の重み、係数

説明変数の重みは`coef_`、係数は`intercept_`に保存されている。[参考ドキュメント](http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [20]:
regr.coef_

array([  8.67095336,  39.079629  ,  51.44432004,  72.12474621,
        12.40113394,  42.90093901,  68.39014667,  62.18786223,
         7.45470731,  32.43973985])

In [21]:
regr.intercept_

-0.19352574416559243

#### 3.3 予測および性能評価

予測値を求めるには`predict`メソッドを用いる。

In [22]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 292.06289379,  189.68325341,   10.48944247,  -23.9953353 ,
        -93.2328178 ])

また`score`メソッドを用いることで実測値と予測値の間の決定係数(r<sup>2</sup>)を求めることができる。

In [23]:
regr.score(X, y)

0.99942942831965143

#### 3.4 ペナルティ項の重み(引数alpha, 講義資料ではλ)を変えてみる

`alpha=0.5`とし説明変数の重みや決定係数が変化するか確認してみる。

In [24]:
regr = Lasso(alpha=0.5)
regr.fit(X, y) 

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [25]:
regr.coef_

array([  9.11881558,  39.63585887,  51.99966865,  72.67897205,
        12.96450284,  43.46200579,  68.97766438,  62.73831398,
         7.92919823,  33.04171313])

In [26]:
regr.intercept_

-0.096763496833000673

In [27]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 296.13466732,  191.98005985,   11.87348133,  -24.29904409,
        -95.99674734])

In [28]:
regr.score(X, y)

0.99985735695339761

わずかであるが説明変数の重みが変化したり、決定係数が大きくなったりした。

#### 3.5 K-fold cross validation

[cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)関数を用いた10-fold cross validationを行い、モデルの性能評価を行う時は以下のようにスクリプトを書けば良い。

In [29]:
regr = Lasso()
cross_val_score(regr, X, y, cv=10)

array([ 0.99939193,  0.99947731,  0.99930002,  0.99935799,  0.99938699,
        0.99951315,  0.99937756,  0.99945147,  0.99938174,  0.99946904])

このようにcross validationを行う時に、以下のように`λ`を変化させて性能の差を比較し、  
最もよい性能の時の`λ`を採用するという判断もできる。  
Lassoの場合、3.6で説明しているように`λ=0`だと警告が出て煩わしいのでalpha=[0.1, 1.0]の間を0.1ずつ増やしてみる。

In [30]:
alphas = [i / 10 for i in range(1, 11)]
alphas

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [31]:
for alpha in alphas:
    regr = Lasso(alpha=alpha)
    scores = cross_val_score(regr, X, y, cv=10)    
    print(f'alpha={alpha}, score_mean={scores.mean()}')

alpha=0.1, score_mean=0.9999941071140442
alpha=0.2, score_mean=0.9999764286427961
alpha=0.3, score_mean=0.9999469645861636
alpha=0.4, score_mean=0.999905714944146
alpha=0.5, score_mean=0.9998526797167436
alpha=0.6, score_mean=0.9997878589039567
alpha=0.7, score_mean=0.9997112525057851
alpha=0.8, score_mean=0.9996228605222284
alpha=0.9, score_mean=0.999522682953287
alpha=1.0, score_mean=0.9994107197989608


今回の例では、`alpha=0.1`のときが最も性能が良い。

#### 3.6 LASSOとOrdinary Least Squaresの関係について

`alpha=0`の時、LASSOはOrdinary Least Squares (OLS) と全く同等のモデルになる。

##### 3.6.1 LASSOによる解析

In [32]:
regr = Lasso(alpha=0)
regr.fit(X, y) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:470: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)


Lasso(alpha=0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

上記の赤枠ででている英文は、`alpha=0`の時、LASSOはOLSと同等であり  
scikit-learnでOLSを行うためのクラスである[LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)を用いたほうが良いと警告している。

In [33]:
regr.coef_

array([  9.5666778 ,  40.19208874,  52.55501726,  73.2331979 ,
        13.52787175,  44.02307257,  69.56518208,  63.28876574,
         8.40368915,  33.64368641])

In [34]:
regr.intercept_

-1.2495004124701836e-06

In [35]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.20644086,  194.27686629,   13.25752019,  -24.60275289,
        -98.76067687])

In [36]:
regr.score(X, y)

0.99999999999996003

##### 3.6.2 OLSによる解析

In [37]:
regr = LinearRegression()
regr.fit(X, y) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [38]:
regr.coef_

array([  9.56666472,  40.19207697,  52.55502375,  73.2332176 ,
        13.52787244,  44.02306615,  69.56518971,  63.28876932,
         8.40368739,  33.64368616])

In [39]:
regr.intercept_

1.7763568394002505e-15

In [40]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

In [41]:
regr.score(X, y)

1.0

おおよそ`λ=0`でLASSOを行ったときと同じ結果が得られた。  
誤差があるのは、LASSOのアルゴリズムでは反復して最適解に収束しているのが原因だと考えられる。

---

### 4. Elastic Net

#### 4.1 機械学習

[Elastic Net](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)クラスのコンストラクタの引数`alpha`が講義資料の`λ`に相当する。今回は`alpha=1`として学習を行ってみる。

In [42]:
regr = ElasticNet(alpha=1.0)
regr.fit(X, y) 

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

#### 4.2 説明変数の重み、係数

説明変数の重みは`coef_`、係数は`intercept_`に保存されている。[参考ドキュメント](http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [43]:
regr.coef_

array([  7.68786731,  25.30575612,  33.5610656 ,  45.63534789,
         7.92149813,  28.35731918,  42.26315229,  40.29762631,
         6.82831102,  20.35875323])

In [44]:
regr.intercept_

-3.4248048262101056

#### 4.3 予測および性能評価

予測値を求めるには`predict`メソッドを用いる。

In [45]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([  1.84164441e+02,   1.18182663e+02,   1.40287588e-01,
        -1.95258994e+01,  -6.90877107e+01])

また`score`メソッドを用いることで実測値と予測値の間の決定係数(r<sup>2</sup>)を求めることができる。

In [46]:
regr.score(X, y)

0.86165994113444722

#### 4.4 ペナルティ項の重み(引数alpha, 講義資料ではλ)を変えてみる

`alpha=0.5`とし説明変数の重みや決定係数が変化するか確認してみる。

In [47]:
regr = ElasticNet(alpha=0.5)
regr.fit(X, y) 

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [48]:
regr.coef_

array([  8.66026878,  31.10447973,  40.99875723,  56.26268625,
        10.08117981,  34.54136184,  52.64455734,  49.2613208 ,
         7.68046321,  25.44061581])

In [49]:
regr.intercept_

-2.0807045509356366

In [50]:
regr = ElasticNet(alpha=0.5)
regr.fit(X, y) 

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [51]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 229.13357404,  147.67331505,    4.81268899,  -21.59286071,
        -81.25942788])

In [52]:
regr.score(X, y)

0.94808728479346494

説明変数の重みが変化したり、決定係数が大きくなったりした。

#### 4.5 K-fold cross validation

[cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)関数を用いた10-fold cross validationを行い、モデルの性能評価を行う時は以下のようにスクリプトを書けば良い。

In [53]:
regr = ElasticNet()
cross_val_score(regr, X, y, cv=10)

array([ 0.85950289,  0.85641179,  0.86099879,  0.85842222,  0.85227743,
        0.85777046,  0.86375841,  0.85920357,  0.85915002,  0.85732303])

このようにcross validationを行う時に、以下のように`λ`を変化させて性能の差を比較し、  
最もよい性能の時の`λ`を採用するという判断もできる。  
ElasticNetの場合、4.6で説明しているようにLASSO同様`λ=0`だと警告が出て煩わしいのでalpha=[0.1, 1.0]の間を0.1ずつ増やしてみる。

In [54]:
alphas = [i / 10 for i in range(1, 11)]
alphas

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [55]:
for alpha in alphas:
    regr = ElasticNet(alpha=alpha)
    scores = cross_val_score(regr, X, y, cv=10)    
    print(f'alpha={alpha}, score_mean={scores.mean()}')

alpha=0.1, score_mean=0.996810381188346
alpha=0.2, score_mean=0.9885562733957226
alpha=0.3, score_mean=0.9767724890744482
alpha=0.4, score_mean=0.9625575353684317
alpha=0.5, score_mean=0.9467050507974617
alpha=0.6, score_mean=0.9297927017470242
alpha=0.7, score_mean=0.9122431674834258
alpha=0.8, score_mean=0.8943665279200195
alpha=0.9, score_mean=0.8763904902695916
alpha=1.0, score_mean=0.8584818600329676


今回の例では、`alpha=0.1`のときが最も性能が良い。

#### 4.6 Elastic NetとOrdinary Least Squaresの関係について

`alpha=0`の時、LASSOと同様にElastic NetはOrdinary Least Squares (OLS) と全く同等のモデルになる。

##### 4.6.1 Elastic Netによる解析

In [56]:
regr = ElasticNet(alpha=0)
regr.fit(X, y) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:470: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)


ElasticNet(alpha=0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

上記の赤枠ででている英文は、`alpha=0`の時、Elastic NetはOLSと同等であり  
scikit-learnでOLSを行うためのクラスである[LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)を用いたほうが良いと警告している。

In [57]:
regr.coef_

array([  9.5666778 ,  40.19208874,  52.55501726,  73.2331979 ,
        13.52787175,  44.02307257,  69.56518208,  63.28876574,
         8.40368915,  33.64368641])

In [58]:
regr.intercept_

-1.2495004124701836e-06

In [59]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.20644086,  194.27686629,   13.25752019,  -24.60275289,
        -98.76067687])

In [60]:
regr.score(X, y)

0.99999999999996003

##### 4.6.2 OLSによる解析

In [61]:
regr = LinearRegression()
regr.fit(X, y) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [62]:
regr.coef_

array([  9.56666472,  40.19207697,  52.55502375,  73.2332176 ,
        13.52787244,  44.02306615,  69.56518971,  63.28876932,
         8.40368739,  33.64368616])

In [63]:
regr.intercept_

1.7763568394002505e-15

In [64]:
y_predicted = regr.predict(X)
y_predicted[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

In [65]:
regr.score(X, y)

1.0

おおよそ`λ=0`でElastic Netを行ったときと同じ結果が得られた。  
誤差があるのは、LASSOと同様にElastic Netのアルゴリズムでは反復して最適解に収束しているのが原因だと考えられる。